### Install Dependencies

In [3]:
# !pip install langdetect geoip2

In [30]:
import os
import json
import gzip
import shutil
import pandas as pd
from email.header import decode_header, make_header
from langdetect import detect
import geoip2.database
from bs4 import XMLParsedAsHTMLWarning
import warnings
from bs4 import BeautifulSoup
from collections import defaultdict
import awswrangler as wr
wr.engine.set("python") 
from multiprocessing import Pool
import logging
logging.getLogger('nltk').setLevel(logging.ERROR)

warnings.filterwarnings("ignore", category=XMLParsedAsHTMLWarning)

### Connect to Sendgrid Role

In [33]:
import boto3
from pprint import pprint

sts = boto3.Session().client("sts", region_name="us-east-1")
response = sts.assume_role(
    RoleArn="arn:aws:iam::375084544312:role/mimesample_delegate",
    RoleSessionName="mimesamples-access"
)

ACCESS_KEY = response["Credentials"]["AccessKeyId"]
SECRET_KEY = response["Credentials"]["SecretAccessKey"]
SESSION_TOKEN = response["Credentials"]["SessionToken"]

session = boto3.Session(
    aws_access_key_id=ACCESS_KEY,
    aws_secret_access_key=SECRET_KEY,
    aws_session_token=SESSION_TOKEN
)
s3 = session.client('s3')

# Your bucket name
bucket_name = 'mime-samples-production'

In [34]:
COMPRESSED_DATA_PATH = 'raw_emails/'
JSON_PATH = 'raw_emails_json/'
#create folder if not already
if not os.path.exists(COMPRESSED_DATA_PATH):
    os.makedirs(COMPRESSED_DATA_PATH)
if not os.path.exists(JSON_PATH):
    os.makedirs(JSON_PATH)
# for f in os.listdir(EMAIL_PATH):
#     os.remove(os.path.join(EMAIL_PATH, f))
files_in_compressed_data = os.listdir(COMPRESSED_DATA_PATH)
files_in_json_data = os.listdir(JSON_PATH)

In [28]:
len(files_in_compressed_data), len(files_in_json_data)

(12, 3)

In [10]:
# paginator = s3.get_paginator('list_objects_v2')
# pages = paginator.paginate(Bucket=bucket_name, Prefix='users/50733733/')
file_list = wr.s3.list_objects("s3://mime-samples-production/users/50733733/", boto3_session=session)
for file in file_list:
    print(file)
# pages = paginator.paginate(Bucket=bucket_name, Prefix='json/day=20250412/')

s3://mime-samples-production/users/50733733/4b3544d1-837b-4995-87a6-5d9ad434f216.json
s3://mime-samples-production/users/50733733/c57823f7-4fe1-43a0-a231-ae0fb1a7e433.json
s3://mime-samples-production/users/50733733/cda86c6d-f6c3-474f-9644-29175d574016.json
s3://mime-samples-production/users/50733733/dafed507-8e22-4ad5-ba83-63e753d4778a.json


In [12]:
# for page in pages:
#     for obj in page['Contents']:
for key in file_list:
        user = key.split('/')[1]
        user_base_path = os.path.join(JSON_PATH, 'users', user)
        if not os.path.exists(user_base_path):
            os.mkdir(user_base_path)
        local_filename = os.path.join(user_base_path, key.split('/')[2])  # extract just the filename
        print(f"Downloading: {key} to {local_filename}")
        
        # Download file from S3 to local
        wr.s3.download(key, local_filename, boto3_session=session)
        print(f"Downloaded {local_filename} successfully.")

Downloading: s3://mime-samples-production/users/50733733/4b3544d1-837b-4995-87a6-5d9ad434f216.json to raw_emails_json/users/mime-samples-production
Downloaded raw_emails_json/users/mime-samples-production successfully.
Downloading: s3://mime-samples-production/users/50733733/c57823f7-4fe1-43a0-a231-ae0fb1a7e433.json to raw_emails_json/users/mime-samples-production
Downloaded raw_emails_json/users/mime-samples-production successfully.
Downloading: s3://mime-samples-production/users/50733733/cda86c6d-f6c3-474f-9644-29175d574016.json to raw_emails_json/users/mime-samples-production
Downloaded raw_emails_json/users/mime-samples-production successfully.
Downloading: s3://mime-samples-production/users/50733733/dafed507-8e22-4ad5-ba83-63e753d4778a.json to raw_emails_json/users/mime-samples-production
Downloaded raw_emails_json/users/mime-samples-production successfully.


In [ ]:
"""
1 page = 1000 files
1 file = .gz file --> unzip --> json 
1 json file ~= 2000 emails

json/day=20250412/<filename>.gz --> 1 JSON file --> multiple emails (each line = 1 email) ~ 2000

users/<userid>/<filename>.json --> single email in 1 line = 1 email

Suspended User  Suspension Date
<user_id>       <date>

emails <date> - 3 days to <date> = Phish

- number of users
- emails per date
"""

In [21]:
from preprocessor import PreProcessor

2025-04-22 12:48:20.514791: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [22]:
def process_file(file_path, date):
    local_data = {
        'user_id': [], 'msg_id': [], 'sg_event_id': [], 'subject': []
        # , 'html_file': []
        , 'mime': [], 'email_from': [], 'email_to': [], 'email_date': [],
        'originating_ip': [], 'originating_ip_country': [], 'lang': []
    }
    
    p = PreProcessor()
    # email_body_dir = os.path.join('emails_with_subject_body', date)
    # os.makedirs(email_body_dir, exist_ok=True)

    with open(file_path, 'r') as f:
        for line in f:
            try:
                j = json.loads(line.strip())
                raw_mime = j["raw_mime"]
                # subject_body = p._pull_subject_body(raw_mime)
                
                # File writing handled per-process
                # filename_base = os.path.splitext(os.path.basename(file_path))[0]
                # body_filename = os.path.join(email_body_dir, f"{filename_base}.html")
                # with open(body_filename, 'w') as f_write:
                #     f_write.write(subject_body[1])

                # Subject
                subject = str(make_header(decode_header(j["event"]["payload"]["subject"]))) if j["event"]["payload"]["subject"] else None

                # GeoIP setup per process
                with geoip2.database.Reader('./GeoIP2-Country.mmdb') as reader:
                    originating_ip = j["event"]["payload"]["originating_ip"]
                    try:
                        country = reader.country(originating_ip).country.name
                    except:
                        country = 'N/A'

                # Language detection
                lang = detect(subject) if subject else 'N/A'

                # Append data
                local_data['user_id'].append(j["event"]["payload"]["userid"])
                local_data['msg_id'].append(j["event"]["payload"]["msgid"])
                local_data['sg_event_id'].append(j["event"]["payload"]["sg_event_id"])
            
                local_data['email_from'].append(j["event"]["payload"]["email_from"])
                local_data['email_to'].append(j["event"]["payload"]["email"])
                local_data['email_date'].append(j["event"]["payload"]["date"])
                
                local_data['subject'].append(subject)
                # local_data['html_file'].append(body_filename)
                local_data['mime'].append(raw_mime)
                local_data['originating_ip'].append(j["event"]["payload"]["originating_ip"])
                local_data['originating_ip_country'].append(country)
                local_data['lang'].append(lang)

            except Exception as e:
                print(f"Error in {file_path}: {str(e)}")
    os.remove(file_path)
    return local_data

In [23]:
def process_date_folder(date_folder):
    date = os.path.basename(date_folder)
    files = [os.path.join(date_folder, f) for f in os.listdir(date_folder)
             if os.path.isfile(os.path.join(date_folder, f))]
    
    with Pool() as pool:
        results = pool.starmap(process_file, [(f, date) for f in files])
    
    # Aggregate results
    combined = {k: [] for k in results[0]}
    for result in results:
        for k, v in result.items():
            combined[k].extend(v)
    return combined

In [31]:
from datetime import datetime, timedelta

def get_date_list(start_date_str, end_date_str):
    # Parse the input strings into datetime objects
    start_date = datetime.strptime(start_date_str, '%Y%m%d')
    end_date = datetime.strptime(end_date_str, '%Y%m%d')

    # Generate the list of dates
    date_list = []
    current_date = start_date
    while current_date <= end_date:
        date_list.append(current_date.strftime('%Y%m%d'))
        current_date += timedelta(days=1)
    
    return date_list

# Example usage
start = '20250320'
end = '20250404'
dates = get_date_list(start, end)

In [ ]:

for d in dates[::-1]:
    date_base_path = os.path.join(COMPRESSED_DATA_PATH, d)
    os.makedirs(date_base_path, exist_ok=True)
    
    json_date_base_path = os.path.join(JSON_PATH, 'dates', d)
    os.makedirs(json_date_base_path, exist_ok=True)

    # paginator = s3.get_paginator('list_objects_v2')
    # pages = paginator.paginate(Bucket=bucket_name, Prefix=f'json/day={d}/')
    location = f"s3://mime-samples-production/json/day={d}"
    keys = wr.s3.list_objects(location, boto3_session=session)
    for key in keys:
        local_filename = os.path.join(date_base_path, key.split('/')[-1])  # extract just the filename
        print(f"Downloading: {key} to {local_filename}")
        
        # Download file from S3 to local
        # s3.download_file(bucket_name, key, local_filename)
        wr.s3.download(key, local_filename, boto3_session=session)
        print(f"Downloaded {local_filename} successfully.")

        # Unzip to JSON
        with gzip.open(local_filename, 'rb') as f_in:
            base_name, _ = os.path.splitext(key.split('/')[-1])
            filepath_out = os.path.join(json_date_base_path, base_name + '.json')
            with open(filepath_out, 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)
        os.remove(local_filename)
        
        print(f"Unzipped file saved to: {filepath_out}")

    processed = process_date_folder(json_date_base_path)
        
    df = pd.DataFrame(processed)
    df = df.sort_values(['user_id', 'sg_event_id'])

    csv_output = 'csv_files/wo_label/date_wise/'
    os.makedirs(csv_output, exist_ok=True)

    output_filename = os.path.join(csv_output, f'data_woLabel_{d}.csv')
    df.to_csv(output_filename, index=False)
    print(f"\n\nProcessed CSV with DF of size {len(df)} and saved to {output_filename}!\n")

Downloading: s3://mime-samples-production/json/day=20250404/email-content-raw-production-1-2025-04-04-00-10-43-2f4655c2-c8f4-4471-8e2b-6b9065d1e36c.gz to raw_emails/20250404/email-content-raw-production-1-2025-04-04-00-10-43-2f4655c2-c8f4-4471-8e2b-6b9065d1e36c.gz
Downloaded raw_emails/20250404/email-content-raw-production-1-2025-04-04-00-10-43-2f4655c2-c8f4-4471-8e2b-6b9065d1e36c.gz successfully.
Unzipped file saved to: raw_emails_json/dates/20250404/email-content-raw-production-1-2025-04-04-00-10-43-2f4655c2-c8f4-4471-8e2b-6b9065d1e36c.json
Downloading: s3://mime-samples-production/json/day=20250404/email-content-raw-production-1-2025-04-04-00-10-56-86b79dc6-e415-4df2-a4a6-fcafe8e3eaf0.gz to raw_emails/20250404/email-content-raw-production-1-2025-04-04-00-10-56-86b79dc6-e415-4df2-a4a6-fcafe8e3eaf0.gz
Downloaded raw_emails/20250404/email-content-raw-production-1-2025-04-04-00-10-56-86b79dc6-e415-4df2-a4a6-fcafe8e3eaf0.gz successfully.
Unzipped file saved to: raw_emails_json/dates/2025

In [36]:
df.head(), df.shape

(      user_id                                             msg_id  \
 236  35459534  -h2aAimDR4OLypvFDN1mZQ.recvd-7c497566f-fz9wr-1...   
 21   35459534  6iZejZ1CSVuIyMctJWs9YA.recvd-7c497566f-vxxrj-1...   
 286  35459534  8CDbYFszRgqdvmqNpixCQQ.recvd-7c497566f-b6nlg-1...   
 419  35459534  MCxFLDVbTLuyFHiEu4V-AA.recvd-7c497566f-gv7jj-1...   
 633  35459534  MFlmznL3Q6usAQriKNjyTQ.recvd-7c497566f-l7v8h-1...   
 
                                            sg_event_id  \
 236  cHJvY2Vzc2VkLTM1NDU5NTM0LS1oMmFBaW1EUjRPTHlwdk...   
 21   cHJvY2Vzc2VkLTM1NDU5NTM0LTZpWmVqWjFDU1Z1SXlNY3...   
 286  cHJvY2Vzc2VkLTM1NDU5NTM0LThDRGJZRnN6UmdxZHZtcU...   
 419  cHJvY2Vzc2VkLTM1NDU5NTM0LU1DeEZMRFZiVEx1eUZIaU...   
 633  cHJvY2Vzc2VkLTM1NDU5NTM0LU1GbG16bkwzUTZ1c0FRcm...   
 
                                                subject  \
 236  CoreTest message for - TestV3SendPersonalizati...   
 21   CoreTest message for - TestV3SendPersonalizati...   
 286  CoreTest message for - TestV3SendPersonalizat

# Snowflake Connection to get Suspended Accounts

In [97]:
import snowflake.connector

conn = snowflake.connector.connect(
    user='monica.tare@sendgrid.com',
    account='sendgrid.us-east-1',
    authenticator= 'externalbrowser',
    warehouse='TRUST_INSIGHTS_STAGING_WH',
    database='TRUST_INSIGHTS_PROD',
    role='MDS_UNMASKED_ACCESS_ROLLUP',  # TRUST_INSIGHTS_DEVELOPER
    schema="trust_insights_prod_schema"
)

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
Going to open: https://sendgrid.us-east-1.snowflakecomputing.com/console/login?login_name=monica.tare%40sendgrid.com&browser_mode_redirect_port=42597&proof_key=LTUzJrXDRnDNaJ8aw%2Fq%2FagjTHoeOdJw25TIm5B7dK%2BM%3D to authenticate...
We were unable to open a browser window for you, please open the url above manually then paste the URL you are redirected to into the terminal.


Enter the URL the SSO URL redirected you to:  http://localhost:42597/?token=7VlJl6LKEt77K-rYS08Vk-Bwuuo-JgEVFBFQdwzJIKMkCPrrH1pldVV13-6-t_u8xTu90ozMiIyILwi_kM9_NUl8dwQFDLP0sYs9oN07kDqZG6b-Y1dfT-6H3b-ePkMrifF8vAIwz1II7jgAyzC1yqtSUJY5HCMIBKnrF6H7UMF7YMHyHnuAaVZ7sRUBJ0vyqlXxH9pviAdcJM78MO3eSdxjN3RJqk-R_f6IGo5GJD4aDocYQZEo0SdRsj2T3i5eZ4_dPApdvnJtaPM0PRdLwsykmmt2OeRKdSDsSMkU7bNwmslzcmvyTqsPYQWkFJZWWj52cRQn79H-PUat0cEYH47R4QNOkbvunXFLA35JQ5uYFI6fI3_sVkU6ziwYwnFqJQCOS2es0fJ83B4d50VWZk4W33Qa-JyUNid1XT_UxENW-AiOohiykeeaE4DE6r4kdXx1rribZEVild-_5yIJ3XvvenQM0jIsT-_8_L66BSEoLpB1n954l0WldQUFNBFWDKqRBQepup9sDcwdfkbeOvn02YVjLfRb4KsCvFzs_l2wKIKOkPaMC0P_U_dVF7hS6mXXJWulWRo6Vhyer6UkgzLI3Ds69rMiLIPkb7OIoRfD96Bx7h2sn37qIu9d-2lDaP_m4X2SFeBTAa17GFg4Sb2YXAEPFO0DAe70lfTY_fSDUr3qrAsrhReQ4PvlD_15lzGQHkGc5cC9h7ewXnz6eYPfyNTTZ-CMpdSJKxgegXIpktxyALxbFsALm3kI2yJs4K2sgPOPQEDeuvdh-ZwNLvTb3vEPAWoB-PQOlmcrhhVX4Kmp4HkmoPS8j7tw6dOWNsvYSjzMkg0pzm25sMyqgZNgNeUfrx69Vb4KXkF-Xn6o0teqetZYYYxghUI_odhINSYigWiGXvUwrzgdEBpdWfL5SJG6

In [98]:
import pandas as pd
def query_snowflake(query):
    df = pd.read_sql(
        query,
        conn
    )
    df.columns = [col.lower() for col in df.columns]
    return df

In [99]:
query = """
select * from edo_dw_prod.acq_maildb.acq_hist_maildb_admin_note
where agent_notes like '%DIRMBS:RElSTUJT%'
    and created_at >= dateadd(month, -2, current_timestamp)
"""
df = query_snowflake(query)

/tmp/ipykernel_1253/1449233593.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(


In [100]:
df.columns # agent_notes, account column = user_id

Index(['etl_acq_row_num', 'etl_src_id', 'etl_src_abbrv', 'etl_src_table_name',
       'etl_src_start_date', 'etl_src_end_date', 'etl_uid_code', 'etl_md5',
       'etl_ins_run_id', 'etl_upd_run_id', 'etl_ins_date', 'etl_upd_date',
       'etl_del_date', 'id', 'user_id', 'agent_id', 'admin_notes_category_id',
       'package_id', 'package_name', 'parent_id', 'linked_note_id',
       'agent_notes', 'created_at', 'updated_at'],
      dtype='object')

In [101]:
users = df['user_id'].unique()

In [102]:
len(users) # unique accounts

641

In [103]:
users

array([51377476, 51444819, 51444353, 51445382, 51447082, 51446010,
       51446574, 51263266, 50968804, 51701167, 50365254, 51239799,
       51127330, 51360496, 51205267,  7955388, 51318199, 51435719,
       51032078, 51006690, 51005788, 51004482, 51010331, 51008317,
       51011704, 51009335, 25609331, 51004908, 51052107, 51052134,
       51062097, 51052153,  3956193, 50946716, 51058680, 51026046,
       51068494, 51052161, 51066333, 51052114, 51052141, 51052122,
       51029092, 50835760, 51063142, 50783561, 51052135, 51059551,
       51006206, 51063723, 51052169, 51029106, 43471765, 51052150,
       51029112, 51067393, 51058238, 51052128, 51062467, 51081361,
       50129380, 51394856, 51819232, 51680176, 51793882, 51773379,
       51753452, 51800414, 50407563, 51230484, 51218412, 51074075,
       50808968, 51205790, 51054841, 51054441, 51054631, 46885400,
       51300960, 46972862, 51052000, 51300956, 46534251, 51300965,
       50797403, 51457565, 51086735, 51347649, 50794444, 50777